In [3]:
# !pip install surprise

In [4]:
# !pip uninstall numpy -y
# !pip install numpy==1.26.0

In [5]:
import pandas as pd
import numpy as np
from surprise import *

# Read Data

In [6]:
df = pd.read_csv("./Products_ThoiTrangNam_rating_raw.csv",sep='\t')
df

,product_id,user_id,user,rating
0,190,1,karmakyun2nd,5
1,190,2,tranquangvinh_vv,5
2,190,3,nguyenquoctoan2005,5
3,190,4,nguyenthuyhavi,5
4,190,5,luonganh5595,5
...,...,...,...,...
1024477,171107,378659,phantom061212,4
1024478,171107,225341,donggiang_94,4
1024479,171107,650634,ethanhunt94,3
1024480,171107,650635,nguyendiu.3.3.95,2


In [7]:
df.head()

,product_id,user_id,user,rating
0,190,1,karmakyun2nd,5
1,190,2,tranquangvinh_vv,5
2,190,3,nguyenquoctoan2005,5
3,190,4,nguyenthuyhavi,5
4,190,5,luonganh5595,5


# Load data for surPrise

In [8]:
reader = Reader(rating_scale=(df.rating.min(), df.rating.max()))

data = Dataset.load_from_df(
    df[['user_id', 'product_id', 'rating']],
    reader
)

**Split Data**

In [9]:
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=0.25)

# Build Model

In [10]:
algo = SVD()
algo.fit(trainset)

# Eval on test set

In [11]:
predictions = algo.test(testset)

In [12]:
from surprise import accuracy
print('RMSE on test_set: ',accuracy.rmse(predictions))

RMSE: 0.8805
RMSE on test_set:  0.8804797851665349


# Predict for specific user

In [13]:
user_id = 20

In [14]:
product_id_score = df[['product_id']].drop_duplicates().reset_index(drop=True)
product_id_score

,product_id
0,190
1,191
2,192
3,193
4,194
...,...
31262,171103
31263,171104
31264,171105
31265,171106


In [15]:
product_id_score['Score_Prediction']= product_id_score['product_id'].apply(lambda x: algo.predict(user_id, x).est)
product_id_score = product_id_score.sort_values(by=['Score_Prediction'], ascending=False).drop_duplicates()
product_id_score.head()

,product_id,Score_Prediction
7574,23185,5.0
11992,23629,5.0
21937,13727,5.0
27269,171739,5.0
14372,251458,5.0


In [16]:
import pickle

# Save the trained model to a file
model_filename = 'surprise_svd_model.pkl'
with open(model_filename, 'wb') as file:
    pickle.dump(algo, file)

In [26]:
#dump also the df
with open('user_rating_df.pkl', 'wb') as file:
    pickle.dump(df, file)

In [17]:
from importlib import reload
import utils

# Reload the utils module to get the latest changes
# by this I don't need to restart the kernel everytime I modify the function in Utils
reload(utils)

# Now try your function call again
from utils import get_recommendations_surprise

In [24]:
recommendations = get_recommendations_surprise(df=product_id_score,surprise=algo,
                user_id=200,
                nums=1000
                )


In [25]:
recommendations

,product_id,Score_Prediction
27569,172050,5.000000
2485,211058,5.000000
17190,112548,5.000000
29062,173386,5.000000
24916,14950,5.000000
...,...,...
3392,21590,4.877353
21604,13387,4.876982
24987,141021,4.876803
17400,11438,4.876757
